# An Introduction to Bayesian Optimization II: Utility Functions and Gaussian Process Kernels

_Prepared by Ritz Aguilar for Thrill Summer School at Toulon, France, October 2024_

#### Objectives
- Differentiate the effects of using different utility/acquistion functions for BO.
- Use varying kernels for the Gaussian Process, e.g. Constant, Matern, RBF, etc.
- 

**Utility functions**, a.k.a. acquisition functions, guide the selection of the next point to evaluate in a black-box optimization problem. They balance two completing goals:
- _Exploration_: Searches unknown regions of the input space where the objective function has high uncertainty
- _Exploitation_: Focuses on regions where the model predicts high values based on the knowledge it has gathered so far.
    

*__Recall__*: In Bayesian Optimization (BO), a surrogate model (often a Gaussian Process) is fitted to the known data points -- evaluations of the objective function. This model provides both:
- A _mean prediction_, which is the model's estimate of the objective function at any given point.
- A _variance_ or _uncertainty estimate_, representing the confidence the model has in its prediction.

The acquisition (or utility) function uses these predictions to decide the next point to evaluate by assigning a score (utility value) to each possible point. The point with the highest score is selected as the next one to query.

#### Examples of Commonly-used Utility Functions:

1. **Expected Improvement (EI)**: Measures the expected amount of improvement over the best-observed value so far, balancing both uncertainty and the predicted value. Points with high potential for improvement are favored. The function is given by: $$-\text{EI}(x) = -\mathbb{E} [f(x) - f(\hat{x})].$$ _Note: A full implementation of this is found in Notebook Part I._

2. **Upper Confidence Bound (UCB)**: Incorporates both the mean and uncertainty by using a confidence bound to prioritize points with either high predicted values or high uncertainty. It is given by: $$\text{UCB}(x) = \mu_{GP}(x) + \kappa \sigma_{GP}(x).$$

3. **Probability of Improvement (PI)**: Measures the probability that a given point will improve over the current best value. It is given by: $$-\text{PI}(x) = -P(f(x) \geq f(\hat{x}) + \kappa).$$

#### Key Concept

The acquisition function is what transforms the BO process from a passive fitting of the objective function to an active search strategy. It enables efficient exploration of the search space while avoiding unnecessary evaluations, especially in expensive-to-evaluate problems.

#### Summary

The acquisition function in BO is the mathematical function that helps decide where to sample next by assigning a score to each candidate point, balancing the tradeoff between exploration (uncertain areas) and exploitation (promising areas).